# Project Description

# Part 1: Data Preparation

In [288]:
import pandas as pd

df = pd.read_csv('KAG_energydata_complete.csv', index_col=0)
print(df.shape)
print(df.columns)
print(df.head(10))
df.info()

(19735, 28)
Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')
                     Appliances  lights         T1       RH_1     T2  \
date                                                                   
2016-01-11 17:00:00          60      30  19.890000  47.596667  19.20   
2016-01-11 17:10:00          60      30  19.890000  46.693333  19.20   
2016-01-11 17:20:00          50      30  19.890000  46.300000  19.20   
2016-01-11 17:30:00          50      40  19.890000  46.066667  19.20   
2016-01-11 17:40:00          60      40  19.890000  46.333333  19.20   
2016-01-11 17:50:00          50      40  19.890000  46.026667  19.20   
2016-01-11 18:00:00          60      50  19.890000  45.766667  19.20   
2016-01-11 18:10:00          60      50  19.856667 

# Part 2: Feature Engineering

## 1. Time features

In [289]:
import numpy as np

In [290]:
df = df.copy()
df.index = pd.to_datetime(df.index)

In [291]:
# extract basic time features
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek  
df['month'] = df.index.month
df['day_of_month'] = df.index.day
df['day_of_year'] = df.index.dayofyear

In [292]:
# weekend or not
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)

In [293]:
conditions = [
    (df['hour'] >= 6) & (df['hour'] < 12),
    (df['hour'] >= 12) & (df['hour'] < 18),
    (df['hour'] >= 18) & (df['hour'] < 24)
]

choices = [0, 1, 2]  # morning, afternoon, evening

df['time_period'] = np.select(conditions, choices, default=3) # night


In [294]:
# to encode periodic time features

"""
In order to capture periodic patterns, we apply cyclical encoding using sine and cosine transformations. 
This converts discrete time units into continuous circular coordinates.
It maintains the proximity between adjacent periods and eliminating artificial discontinuities at period boundaries.

"""

# hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# week
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# day of month
df['day_of_month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / 30)
df['day_of_month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / 30)

# day of year
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 366)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 366)

## 2. Temperature features

In [295]:
df["T_indoor_avg"] = df[df.filter(like='T').columns.drop('T6','T_out')].mean(axis=1) # average temperature of 8 rooms
df["T_indoor_std"] = df[df.filter(like='T').columns.drop('T6','T_out')].std(axis=1) # standard error of temperature of 8 rooms
df["T_indoor_max"] = df[df.filter(like='T').columns.drop('T6','T_out')].max(axis=1) # maximum temperature of 8 rooms
df["T_indoor_min"] = df[df.filter(like='T').columns.drop('T6','T_out')].min(axis=1) # minimum temperature of 8 rooms
df["T_indoor_range"] = df["T_indoor_max"] - df["T_indoor_min"] # the range of indoor temperature
df['T_indoor_change'] = df['T_indoor_avg'].diff() # change rate of indoor temperature

In [296]:
# the difference between indoor and outdoor temperatures
df["T_diff_1"] = df["T_indoor_avg"] - df["T6"]
df["T_diff_2"] = df["T_indoor_avg"] - df["T_out"]

## 3. Humidity features

In [297]:
df["RH_indoor_avg"] = df[df.filter(like='RH').columns.drop('RH_6','RH_out')].mean(axis=1) # average humidity of 8 rooms
df["RH_indoor_std"] = df[df.filter(like='RH').columns.drop('RH_6','RH_out')].std(axis=1) # standard error of humidity of 8 rooms
df["RH_indoor_max"] = df[df.filter(like='RH').columns.drop('RH_6','RH_out')].max(axis=1) # maximum humidity of 8 rooms
df["RH_indoor_min"] = df[df.filter(like='RH').columns.drop('RH_6','RH_out')].min(axis=1) # minimum humidity of 8 rooms
df["RH_indoor_range"] = df["RH_indoor_max"] - df["RH_indoor_min"] # the range of indoor humidity
df['RH_indoor_change'] = df['RH_indoor_avg'].diff() # change rate of indoor humidity

In [298]:
# the difference between indoor and outdoor humidity
df["RH_diff_1"] = df["RH_indoor_avg"] - df["RH_6"]
df["RH_diff_2"] = df["RH_indoor_avg"] - df["RH_out"]

In [299]:
pip install metpy

Note: you may need to restart the kernel to use updated packages.


In [300]:
# comfort measures
import metpy.calc as mpcalc
from metpy.calc import heat_index
from metpy.units import units
temp = df["T_indoor_avg"].values * units.degC
humidity = df["RH_indoor_avg"].values * units.percent

# heat index
df["heat_index"] = heat_index(temp, humidity)

# dewpoint
df["dewpoint"] = mpcalc.dewpoint_from_relative_humidity(temp, humidity)

## 4. Weather measures

In [301]:
# weather comprehensive measure
df["weather_com"] = df["T_out"].values * df["RH_out"].values *0.01 * df["Windspeed"].values

In [302]:
# change rate
df['T_out_change'] = df['T_out'].diff()
df['Press_change'] = df['Press_mm_hg'].diff()
df['Windspeed_change'] = df['Windspeed'].diff()

## 5. Lag features

In [303]:
# appliance lag
target_col='Appliances'
df[f'{target_col}_lag1'] = df[target_col].shift(1)  
df[f'{target_col}_lag2'] = df[target_col].shift(2)  
df[f'{target_col}_lag3'] = df[target_col].shift(3)
df[f'{target_col}_lag6'] = df[target_col].shift(6)
df[f'{target_col}_lag144'] = df[target_col].shift(144)

In [304]:
# lights lag
df['lights_lag1'] = df['lights'].shift(1)  
df['lights_lag2'] = df['lights'].shift(2)  
df['lights_lag3'] = df['lights'].shift(3)
df['lights_lag4'] = df['lights'].shift(6)
df['lights_lag5'] = df['lights'].shift(144)

In [305]:
# weather lag
df['T_indoor_lag1'] = df['T_indoor_avg'].shift(1)
df['T_indoor_lag6'] = df['T_indoor_avg'].shift(6)

df['T_out_lag1'] = df['T_out'].shift(1)
df['T_out_lag6'] = df['T_out'].shift(6)

## 6. Rolling features

In [306]:
# appliance rolling
df[f'{target_col}_rolling_mean_6'] = df[target_col].rolling(window=6, min_periods=1).mean().shift(1)
df[f'{target_col}_rolling_mean_18'] = df[target_col].rolling(window=18, min_periods=1).mean().shift(1)
df[f'{target_col}_rolling_max_6'] = df[target_col].rolling(window=6, min_periods=1).max().shift(1)
df[f'{target_col}_rolling_min_6'] = df[target_col].rolling(window=6, min_periods=1).min().shift(1)
df[f'{target_col}_rolling_std_18'] = df[target_col].rolling(window=18, min_periods=1).std().shift(1)
df[f'{target_col}_rolling_std_36'] = df[target_col].rolling(window=36, min_periods=1).std().shift(1)
df[f'{target_col}_MA_3'] = df[target_col].rolling(window=3, min_periods=1).mean().shift(1)
df[f'{target_col}_MA_12'] = df[target_col].rolling(window=12, min_periods=1).mean().shift(1)

In [307]:
# lights rolling
df['lights_rolling_mean_6'] = df['lights'].rolling(window=6, min_periods=1).mean().shift(1)
df['lights_rolling_mean_18'] = df['lights'].rolling(window=18, min_periods=1).mean().shift(1)
df['lights_rolling_max_6'] = df['lights'].rolling(window=6, min_periods=1).max().shift(1)
df['lights_rolling_min_6'] = df['lights'].rolling(window=6, min_periods=1).min().shift(1)
df['lights_rolling_std_18'] = df['lights'].rolling(window=18, min_periods=1).std().shift(1)
df['lights_rolling_std_36'] = df['lights'].rolling(window=36, min_periods=1).std().shift(1)
df['lights_MA_3'] = df['lights'].rolling(window=3, min_periods=1).mean().shift(1)
df['lights_MA_12'] = df['lights'].rolling(window=12, min_periods=1).mean().shift(1) 

In [308]:
# weather rolling
df['T_indoor_rolling_mean_6'] = df['T_indoor_avg'].rolling(window=6, min_periods=1).mean().shift(1)
df['T_indoor_rolling_max_6'] = df['T_indoor_avg'].rolling(window=6, min_periods=1).max().shift(1)
df['T_indoor_rolling_min_6'] = df['T_indoor_avg'].rolling(window=6, min_periods=1).min().shift(1)
df['T_indoor_rolling_std_6'] = df['T_indoor_avg'].rolling(window=6, min_periods=1).std().shift(1)

df['T_out_rolling_mean_6'] = df['T_out'].rolling(window=6, min_periods=1).mean().shift(1)
df['T_out_rolling_std_6'] = df['T_out'].rolling(window=6, min_periods=1).std().shift(1)

# Part 3: Feature Analysis

In [309]:
# split train and test dataset
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Appliances"]) 
y = df["Appliances"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [310]:
train_df = X_train.copy()
train_df["Appliances"] = y_train

In [311]:
# correlation-based feature selection
corr_matrix = train_df.corr()
corr_with_target = corr_matrix[target_col].drop(target_col, errors='ignore')

k = 60
top_k = corr_with_target.abs().sort_values(ascending=False)[:k].index

selected_target_corr = corr_with_target[top_k].sort_values(ascending=False)
for i, (feature, corr) in enumerate(selected_target_corr.items(), 1):
    sign = "+" if corr > 0 else "-"
    print(f"{i:2d}. {feature:35s}: {corr:7.4f} ({sign})")

 1. Appliances_lag1                    :  0.7568 (+)
 2. Appliances_MA_3                    :  0.6511 (+)
 3. Appliances_rolling_max_6           :  0.6080 (+)
 4. Appliances_rolling_mean_6          :  0.5862 (+)
 5. Appliances_MA_12                   :  0.5530 (+)
 6. Appliances_lag2                    :  0.5400 (+)
 7. Appliances_rolling_std_18          :  0.5172 (+)
 8. Appliances_rolling_mean_18         :  0.5085 (+)
 9. Appliances_rolling_min_6           :  0.4534 (+)
10. Appliances_lag3                    :  0.4391 (+)
11. Appliances_rolling_std_36          :  0.3947 (+)
12. Appliances_lag6                    :  0.3198 (+)
13. hour                               :  0.2260 (+)
14. lights                             :  0.2176 (+)
15. T_indoor_change                    :  0.2174 (+)
16. Appliances_lag144                  :  0.2137 (+)
17. lights_MA_3                        :  0.2100 (+)
18. lights_rolling_max_6               :  0.2090 (+)
19. lights_lag1                        :  0.20

1. The strong correlation between the target variable and its lagged and rolling statistics indicates autocorrelation, which is consistent with the persistent nature of residential appliance usage.  
2. "Lights" feature is a behavorial proxy rather than a main driver as its correlation score is not high.

In [312]:
# Bottom correlation features
all_correlations = corr_with_target.abs().sort_values(ascending=False)
print(all_correlations.tail(20))

T_indoor_std       0.028703
day_of_month       0.026763
day_of_week        0.024823
T_indoor_min       0.023654
T9                 0.022225
T_out_change       0.019856
Tdewpoint          0.016903
day_of_year_sin    0.013728
day_of_year        0.013239
dewpoint           0.012193
day_of_year_cos    0.011220
rv2                0.007158
rv1                0.007158
month_cos          0.007061
month              0.006134
Visibility         0.005451
RH_4               0.004894
is_weekend         0.002485
RH_5               0.002350
month_sin          0.001511
Name: Appliances, dtype: float64


AS the correlation results between rv1/rv2 and the target are approximately zero, we can delete these two random variables.

In [313]:
X_train = X_train.drop(columns=["rv1","rv2", "T1", "T2", "T3", "T4", "T5", "T7", "T8", "T9", "RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_7", "RH_8", "RH_9"]) 

# Part 4: Train model

## Catboost

In [314]:
all_features = X_train.columns  

feature_categories = {
    'Lights': [f for f in all_features if 'lights' in f],
    'Appliances': [f for f in all_features if 'Appliances' in f],
    'Indoor': [
    f for f in all_features 
    if (('T_indoor' in f or 'T' in f or 'RH' in f or 'RH_indoor' in f or f == 'dewpoint') and ('rolling' not in f and 'lag' not in f)
        and f not in ['T_out', 'RH_out', 'Tdewpoint'])
],

    'Outdoor/Weather': [f for f in all_features if f in ['T_out', 'RH_out', 'Press_mm_hg', 'Press_change', 
                                                         'Windspeed', 'Windspeed_change', 'Visibility',
                                                         'Tdewpoint', 'heat_index', 'weather_com']],
    'Time': [f for f in all_features if f in ['hour', 'day_of_week', 'month', 'day_of_month', 'day_of_year',
                                               'is_weekend', 'time_period', 'hour_sin', 'hour_cos',
                                               'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
                                               'day_of_month_sin', 'day_of_month_cos', 'day_of_year_sin', 'day_of_year_cos']],
    'lagging/rolling': [f for f in all_features if (('lag' in f or 'rolling' in f) and 'Appliances' not in f and 'lights' not in f)]
                                    
}

for cat, feats in feature_categories.items():
    print(f"{cat}: {len(feats)} features")


Lights: 14 features
Appliances: 13 features
Indoor: 20 features
Outdoor/Weather: 10 features
Time: 17 features
lagging/rolling: 10 features


In [315]:
from catboost import CatBoostRegressor

In [316]:
model_cat = CatBoostRegressor(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    verbose=False
)
model_cat.fit(X_train, y_train)

In [317]:
#select features using feature importance
selected_features_by_category = {}
all_selected_features_set = set()

for group_name, features in feature_categories.items():
    num_to_select = max(1, len(features) // 2)  
    selected = model_cat.select_features(
        X=X_train,
        y=y_train,
        features_for_select=features,
        num_features_to_select=num_to_select,  
        algorithm='RecursiveByShapValues',
        steps=4,
        train_final_model=False,
        logging_level='Silent'
    )

    unique_feats = [f for f in selected['selected_features'] if f not in all_selected_features_set]
    selected_features_by_category[group_name] = unique_feats
    all_selected_features_set.update(unique_feats)

selected_features_by_category_names = {}
for category, feats in selected_features_by_category.items():
    feat_names = [X_train.columns[f] if isinstance(f, int) else f for f in feats]
    selected_features_by_category_names[category] = feat_names

for category, feat_names in selected_features_by_category_names.items():
    print(f"{category}: {feat_names}")

Lights: ['lights', 'lights_lag1', 'lights_rolling_mean_6', 'lights_rolling_std_18', 'lights_lag5', 'lights_rolling_std_36', 'lights_MA_12']
Appliances: ['Appliances_MA_3', 'Appliances_MA_12', 'Appliances_rolling_mean_6', 'Appliances_lag1', 'Appliances_rolling_std_18', 'Appliances_lag2']
Indoor: ['T_indoor_change', 'T_diff_1', 'RH_indoor_avg', 'RH_indoor_std', 'RH_indoor_change', 'RH_diff_1', 'RH_diff_2', 'dewpoint', 'T_out_change', 'T_indoor_max']
Outdoor/Weather: ['Press_mm_hg', 'Windspeed', 'Visibility', 'Press_change', 'Windspeed_change']
Time: ['hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'hour', 'day_of_month', 'day_of_year', 'hour_sin']
lagging/rolling: ['T_indoor_lag1', 'T_indoor_lag6', 'T_out_lag6', 'T_indoor_rolling_std_6', 'T_out_rolling_std_6']


In [318]:
selected_feature_names_cat = []
for feat_names in selected_features_by_category_names.values():
    selected_feature_names_cat.extend(feat_names)
    
X_train_selected_features = X_train[selected_feature_names_cat]
model_cat.fit(X_train_selected_features, y_train)

In [319]:
# final feature selection using permutation importance
# calculate pfi
from sklearn.inspection import permutation_importance
pfi_result = permutation_importance(
    model_cat, 
    X_train_selected_features, 
    y_train, 
    n_repeats=10,  
    random_state=42,
    scoring='neg_root_mean_squared_error'  
)
feature_importance_df = pd.DataFrame({
    'feature': X_train_selected_features.columns,
    'importance': pfi_result.importances_mean
}).sort_values(by='importance', ascending=False)


In [320]:
# PFI importance ranking by category
pfi_by_category = {}

for category, feat_names in selected_features_by_category_names.items():
    df_cat = feature_importance_df[
        feature_importance_df['feature'].isin(feat_names)
    ].sort_values(by='importance', ascending=False)

    pfi_by_category[category] = df_cat

In [321]:
final_features_by_category = {}
used_features = set()

# assign one feature for each category
for category, df_cat in pfi_by_category.items():
    f = df_cat.iloc[0]['feature']
    final_features_by_category[category] = [f]
    used_features.add(f)



In [322]:
MAX_FEATURES = 20
current_count = len(used_features)

remaining_df = feature_importance_df[
    ~feature_importance_df['feature'].isin(used_features)
]

for _, row in remaining_df.iterrows():
    if current_count >= MAX_FEATURES:
        break

    feature = row['feature']

    for category, feat_list in selected_features_by_category_names.items():
        if feature in feat_list:
            final_features_by_category[category].append(feature)
            used_features.add(feature)
            current_count += 1
            break


In [323]:
for category, feats in final_features_by_category.items():
    print(f"{category} ({len(feats)}):")
    for f in feats:
        print(f"  - {f}")


Lights (1):
  - lights
Appliances (6):
  - Appliances_lag1
  - Appliances_lag2
  - Appliances_MA_3
  - Appliances_MA_12
  - Appliances_rolling_mean_6
  - Appliances_rolling_std_18
Indoor (6):
  - RH_indoor_change
  - T_indoor_max
  - T_indoor_change
  - RH_diff_1
  - RH_diff_2
  - RH_indoor_avg
Outdoor/Weather (1):
  - Press_mm_hg
Time (2):
  - hour_sin
  - hour_cos
lagging/rolling (4):
  - T_out_rolling_std_6
  - T_indoor_lag1
  - T_indoor_rolling_std_6
  - T_out_lag6


In [324]:
final_feature_names = []
for feats in final_features_by_category.values():
    final_feature_names.extend(feats)


In [325]:
X_train_final = X_train[final_feature_names]

In [326]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

In [327]:
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [300, 500],
}


In [328]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

cat_model = CatBoostRegressor(
    loss_function='RMSE',
    random_seed=42,
    verbose=0
)

grid_search = GridSearchCV(
    estimator=cat_model,
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train_final, y_train)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001875DA03790>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [300, 500],
                         'l2_leaf_reg': [1, 3, 5],
                         'learning_rate': [0.03, 0.05, 0.1]},
             scoring='neg_root_mean_squared_error')

In [329]:
print("Best RMSE (CV):", -grid_search.best_score_)
print("Best parameters:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_


Best RMSE (CV): 64.11899304089701
Best parameters:
{'depth': 6, 'iterations': 300, 'l2_leaf_reg': 5, 'learning_rate': 0.05}


In [330]:
# using final selected features and best params on test dataset
best_params = {
    'depth': 6,
    'iterations': 300,
    'learning_rate': 0.05,
    'l2_leaf_reg': 5,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'verbose': 0
}
X_test_final  = X_test[final_feature_names]

final_model = CatBoostRegressor(**best_params)
final_model.fit(X_train_final, y_train)

In [331]:
y_test_pred = final_model.predict(X_test_final)

In [332]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test  = mean_absolute_error(y_test, y_test_pred)
r2_test   = r2_score(y_test, y_test_pred)

print(f"Test RMSE: {rmse_test:.3f}")
print(f"Test MAE : {mae_test:.3f}")
print(f"Test R2  : {r2_test:.3f}")

Test RMSE: 57.919
Test MAE : 26.411
Test R2  : 0.595
